In [1]:
import pandas as pd
import datetime
import random

In [2]:
import os

# --- Configuration ---
# You'd typically load the data that includes your shortlisted candidates
# For demonstration, we'll assume a 'shortlisted' flag exists
INPUT_CSV_PATH_FINAL = "./data/resume_job_match_with_gender.csv" # Or the output from your final model prediction
GOOD_MATCH_THRESHOLD = 4

# Mock data for interviewers and their availability
# In a real system, this would come from a database or calendar API
INTERVIEWERS = ["Interviewer A", "Interviewer B", "Interviewer C"]
AVAILABLE_SLOTS_PER_DAY = ["09:00", "10:00", "11:00", "13:00", "14:00", "15:00"]
DAYS_AHEAD = 7 # Look for slots in the next 7 days

# --- Helper Function for generating mock availability ---
def generate_mock_availability(start_date, num_days, interviewers, slots_per_day):
    availability = {}
    current_date = start_date
    for _ in range(num_days):
        date_str = current_date.strftime("%Y-%m-%d")
        availability[date_str] = {}
        for interviewer in interviewers:
            # Simulate random availability for each interviewer and slot
            available_slots = random.sample(slots_per_day, k=random.randint(2, len(slots_per_day)))
            availability[date_str][interviewer] = sorted(available_slots)
        current_date += datetime.timedelta(days=1)
    return availability
# --- Interview Scheduling Function ---
def schedule_interviews(shortlisted_candidates_df, mock_interviewer_availability):
    """
    Schedules interviews for shortlisted candidates based on mock interviewer availability.
    """
    scheduled_interviews = []
    available_interviewers = list(mock_interviewer_availability.keys()) # Dates as keys
    
    print("\n--- Starting Interview Scheduling ---")

    for index, candidate_row in shortlisted_candidates_df.iterrows():
        candidate_id = index # Using DataFrame index as a simple candidate ID for now
        candidate_resume_snippet = candidate_row['resume_cleaned'][:50] + "..."
        
        # Simple availability check: Try to find the first available slot
        found_slot = False
        for date_str, daily_availability in mock_interviewer_availability.items():
            for interviewer, slots in daily_availability.items():
                if slots: # If there are available slots for this interviewer on this day
                    chosen_slot = slots.pop(0) # Take the first available slot and remove it
                    scheduled_interviews.append({
                        "candidate_id": candidate_id,
                        "resume_snippet": candidate_resume_snippet,
                        "date": date_str,
                        "time": chosen_slot,
                        "interviewer": interviewer
                    })
                    print(f"  Scheduled: Candidate ID {candidate_id} (Resume: '{candidate_resume_snippet}') with {interviewer} on {date_str} at {chosen_slot}")
                    found_slot = True
                    break # Move to the next candidate
            if found_slot:
                break # Break from outer loop if slot found for candidate
        
        if not found_slot:
            print(f"  Could not find a slot for Candidate ID {candidate_id} (Resume: '{candidate_resume_snippet}')")

    return scheduled_interviews

# --- Main execution for scheduling ---

# Load data (assuming 'is_good_match' is already created from previous steps)
if not os.path.exists(INPUT_CSV_PATH_FINAL):
    print(f"Error: Input data file not found at '{INPUT_CSV_PATH_FINAL}'. Cannot proceed with scheduling.")
    exit()
try:
    df_final = pd.read_csv(INPUT_CSV_PATH_FINAL)
    df_final['is_good_match'] = (df_final['match_score'] >= GOOD_MATCH_THRESHOLD).astype(int)
    print(f"\nLoaded data for scheduling. Shape: {df_final.shape}")
except Exception as e:
    print(f"Error loading data for scheduling: {e}")
    exit()

# Filter for shortlisted candidates (those predicted as good match, or those with original match_score >= 4)
# For this demo, let's use the 'match_score' directly for shortlisting, or if you had a model,
# you'd use its predictions (e.g., df_final[df_final['predicted_good_match'] == 1])
shortlisted_candidates = df_final[df_final['is_good_match'] == 1]
print(f"Found {len(shortlisted_candidates)} shortlisted candidates.")

if not shortlisted_candidates.empty:
    # Generate mock interviewer availability for demonstration
    start_date = datetime.date.today()
    mock_availability = generate_mock_availability(start_date, DAYS_AHEAD, INTERVIEWERS, AVAILABLE_SLOTS_PER_DAY)
    
    # Run the scheduling function
    final_schedules = schedule_interviews(shortlisted_candidates, mock_availability)
    
    if final_schedules:
        print("\n--- Final Scheduled Interviews ---")
        for schedule in final_schedules:
            print(f"  Candidate: {schedule['candidate_id']} | Interviewer: {schedule['interviewer']} | Date: {schedule['date']} | Time: {schedule['time']}")
    else:
        print("\nNo interviews were scheduled.")
else:
    print("\nNo candidates were shortlisted, so no interviews to schedule.")

print("\nInterview scheduling logic demonstrated.")
print("This can be expanded with more complex rules, calendar APIs, and conflict resolution.")



Loaded data for scheduling. Shape: (10000, 8)
Found 5401 shortlisted candidates.

--- Starting Interview Scheduling ---
  Scheduled: Candidate ID 0 (Resume: 'experienced professional skilled in sql, power bi,...') with Interviewer A on 2025-08-15 at 10:00
  Scheduled: Candidate ID 1 (Resume: 'experienced professional skilled in python, deep l...') with Interviewer A on 2025-08-15 at 13:00
  Scheduled: Candidate ID 2 (Resume: 'experienced professional skilled in wait, git, sys...') with Interviewer B on 2025-08-15 at 09:00
  Scheduled: Candidate ID 3 (Resume: 'experienced professional skilled in return, democr...') with Interviewer B on 2025-08-15 at 10:00
  Scheduled: Candidate ID 4 (Resume: 'experienced professional skilled in rest apis, agi...') with Interviewer B on 2025-08-15 at 11:00
  Scheduled: Candidate ID 8 (Resume: 'experienced professional skilled in as, choice, sq...') with Interviewer B on 2025-08-15 at 13:00
  Scheduled: Candidate ID 12 (Resume: 'experienced professional